In [ ]:
import sys
import tweepy
import time
import csv
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import nltk
from nltk.corpus import stopwords as swords
from wordcloud import WordCloud, STOPWORDS
import twint
import re
from textblob import TextBlob

In [ ]:
#Twitter API token information is saved in a separate file (txt|bak) and read in per line
TWITTER = open("TwiTokens.bak", "r").read().splitlines()
auth = tweepy.OAuthHandler(TWITTER[6], TWITTER[7])
auth.set_access_token(TWITTER[8], TWITTER[9])

api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
target = api.get_user('<TARGET_SCREEN_NAME>')
print(target)

In [ ]:
print(f'ID: {target.id}')
print(f'Username: {target.screen_name}')
print(f'User Bio: {target.description}')
f_date = str(target.created_at)
l_date = str(target.status.created_at)
print(f'Account birthdate: {f_date}')
f_date = f_date.split()
f_date = f_date[0]
f_date = f_date.split('-')
print(f'Last tweet date: {l_date}')
l_date = l_date.split()
l_date = l_date[0]
l_date = l_date.split('-')
delta = dt.date(int(l_date[0]), int(l_date[1]), int(l_date[2])) - dt.date(int(f_date[0]), int(f_date[1]), int(f_date[2]))
print(f'Active Timeline: {delta}')
print(f'Average Tweets Per Day: {target.statuses_count/delta.days}')
print(f'Last Tweet: {target.status.text}')
print(f'Followers Count: {target.followers_count}')
print(f'Friends Count: {target.friends_count}')
print(f'Tweets: {target.statuses_count}')
follow_ratio = target.friends_count/target.followers_count
print(f'Follow Ratio: {follow_ratio}')

In [ ]:
#Only run this for new scrapes. The initial scrape will save output to a textfile. To re-analyze the same target, 
#skip to the next cell and uncomment the first line
target = target.screen_name
with open(f'{target}.txt', 'w+') as target_file:
            for tweet in tweepy.Cursor(api.user_timeline, id=target).items():
                target_file.write(f'{tweet.text}\n')
                target_file.flush()
            target_file.close()

In [ ]:
#target = target.screen_name
with open(f'{target}.txt', 'r') as target_file:
    tweets = target_file.readlines()
    word_count = 0
    rt_list = []
    word_dict = {}
    hashtags = {}
    links = []
    stopwords = set(STOPWORDS)
    table = str.maketrans('', '', string.punctuation)
    for tweet in tweets:
        tweet = tweet.lower()
        curr_string = tweet.split()
        for word in curr_string:
            if 'http:' in word or 'https:' in word:
                links.append(word)
                pass
            if word in stopwords:
                pass
            if word.startswith('#'):
                word = word.translate(table)
                if word not in hashtags:
                    hashtags[word] = 1
                else:
                    hashtags[word] += 1
            word = word.translate(table)
            if word not in word_dict:
                word_dict[word] = 1
            else:
                word_dict[word] += 1
    full_text = " ".join(word for word in word_dict)
    try:
        word_cloud = WordCloud(stopwords=stopwords).generate(full_text)
        plt.imshow(word_cloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()
    except:
        print("No words for the WordCloud apparently")
    target_file.close()
    for word in word_dict.copy():
        if word in swords.words('english'):
            del word_dict[word]
    sort_words = sorted(word_dict.items(), key=lambda x: x[1], reverse=True)
    print("---Top words---")
    for i in sort_words[:20]:
        print(i[0], i[1])
    sorted_hashtags = sorted(hashtags.items(), key=lambda x: x[1], reverse=True)
    print("---Top hashtags---")
    for i in sorted_hashtags[:20]:
        print(i[0], i[1])

In [ ]:
sentiments = []
with open(f'{target}.txt', 'r') as test_file:
    tweets = test_file.readlines()
    for tweet in tweets:
        tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
        """
        # This code block just tested for certain keywords. You can keep it commented out, or uncomment and edit for specific words.
        if 'lol' in tweet:
            print(tweet)
        """
        sent_analysis = TextBlob(tweet)
        if sent_analysis.sentiment.polarity > 0:
            sentiments.append('positive')
        elif sent_analysis.sentiment.polarity == 0:
            sentiments.append('neutral')
        else:
            sentiments.append('negative')
            #print(tweet)
    print(f'---{target}---')
    try:
        print(f'Positive Sentiment Percentage: {sentiments.count("positive")/len(sentiments)}')
        print(f'Negative Sentiment Percentage: {sentiments.count("negative")/len(sentiments)}')
        print(f'Neutral Sentiment Percentage: {sentiments.count("neutral")/len(sentiments)}')
    except:
        print("Potential singularity here")
    test_file.close()